In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import descartes

import geopandas as gpd

from shapely.geometry import Point, Polygon
from shapely.geometry.multipolygon import MultiPolygon

from matplotlib import cm
from matplotlib.colors import ListedColormap, LinearSegmentedColormap
import geoplot as gplt
import geoplot.crs as gcrs
import matplotlib.pyplot as plt

In [ ]:
%matplotlib inline

Карта по районам

In [ ]:
# russia_map = gpd.read_file('draw_picture/RUS_adm/RUS_adm3.shp')
# fig,ax = plt.subplots(figsize = (15,15))
# russia_map.plot(ax = ax)

Карта по регионам

In [ ]:
russia_map = gpd.read_file('draw_picture/RUS_adm/RUS_adm1.shp')
fig,ax = plt.subplots(figsize = (15,15))
russia_map.plot(ax = ax);

In [ ]:
russia_map.geometry.apply(type).unique()
mp = MultiPolygon([russia_map.geometry[0], russia_map.geometry[1]])
#print(mp, type(mp))

In [ ]:
def rectify_poly(p):
    pts = p.exterior.coords[:]
    def angle_corr(angle):
        return angle + 360 if angle < 0 else angle
    pts2 = [(angle_corr(pp[0]), angle_corr(pp[1])) for pp in pts]    
    return Polygon(pts2)

def rectify_multipoly(mp):
    if type(mp) == Polygon:
        return rectify_poly(mp)
    elif type(mp) == MultiPolygon:
        return MultiPolygon([rectify_poly(p) for p in mp])
    
new_geom = russia_map.geometry.apply(rectify_multipoly)

In [ ]:
russia_map2 = russia_map.copy()
russia_map2.geometry = new_geom
crimea_map = gpd.read_file('draw_picture/crimea_9sr/crimea_9sr.shp')

In [ ]:
russia_with_crimea = russia_map2.append({'geometry': crimea_map.geometry[0],
                    'NAME_0': 'Russia', 'ISO': 'RUS',
                    'NAME_1': 'Crimea', 'TYPE_1': 'Respublika',
                    'ENGTYPE_1': 'Republic', 'NL_NAME_1':'Республика Крым',
                    'VARNAME_1': 'Crimea', 'ID_0':188, 'ID_1':91}, ignore_index=True)

In [ ]:
fig,ax = plt.subplots(figsize = (20,20))
russia_with_crimea.plot(ax = ax);

In [ ]:
russia_with_crimea.head()

In [ ]:
# reg_list = pd.read_csv('../indexes/reg_list.csv')
# maplist =  russia_with_crimea.iloc[:,-3]
# rlist = reg_list.iloc[:,1]

# from fuzzywuzzy import fuzz

# k = []
# for i in maplist:
#     lll = [fuzz.ratio(i,j) for j in rlist]
#     k.append(np.argmax(lll))

# rlist2 = rlist[k]
# pdlist=pd.DataFrame([maplist.values, rlist2.values]).transpose()

# # pdlist.iloc[42,:] = 'Москва'
# # pdlist.iloc[81,:] = 'Еврейская автономная область'

# # pdlist.to_csv('map_reg_dict.csv')

In [ ]:
pdlist = pd.read_csv('map_reg_dict.csv', index_col=0)

In [ ]:
russia_with_crimea.iloc[:,-3] =  pdlist.iloc[:,1]

# geo_df[geo_df['Тип месторождения'] == "газовое"].plot(ax = ax, markersize = 20, color = "royalblue", marker = "^", label = "gas field")
# geo_df[geo_df['Тип месторождения'] ==  "нефтяное"].plot(ax = ax, markersize = 20, color = "black", marker = "^", label = "oil field")
# geo_df[geo_df['Тип месторождения'] ==  "нефтегазовое"].plot(ax = ax, markersize = 20, color = "navy", marker = "^", label = "oil and gas field")
# geo_df[geo_df['Тип месторождения'] == ""].plot(ax = ax, markersize = 20, color = "maroon", marker = "o", label = "Nan")
# plt.legend(prop={'size': 15});
# plt.savefig('map_oil_and_gas_fields.png', transparent=True, bbox_inches='tight')

In [ ]:
# russia_with_crimea

In [ ]:
type(russia_with_crimea)

### Preparing Dataset

In [ ]:
glub_1 = pd.read_csv('data_oil_indexes/1_glubina_regions.csv').iloc[7:,[0,2]]

In [ ]:
glub_1.head()

In [ ]:
glub_1 = pd.read_csv('1_efficiency.csv')

In [ ]:
# glub_1

In [ ]:
all_data = pd.read_csv('map_reg_dict.csv', index_col=0)

In [ ]:
all_data = all_data.merge(glub_1, how='outer', left_on='1', right_on='Регион')

all_data = all_data.drop('Регион',axis=1)

In [ ]:
kratnosti_2 = pd.read_csv('data_oil_indexes/2_kratnosti_neft_gas.csv', index_col=0).iloc[:,[1,-2,-1]].replace(np.inf, np.nan)

In [ ]:
maplist =  kratnosti_2.iloc[:,0]
rlist = all_data.iloc[:,1]

from fuzzywuzzy import fuzz

k = []
for i in maplist:
    lll = [fuzz.ratio(i,j) for j in rlist]
    k.append(np.argmax(lll))

rlist2 = rlist[k]

# pd.DataFrame([maplist.values, rlist2.values]).transpose()

kratnosti_2.iloc[:,0] = rlist[k].values


In [ ]:
# kratnosti_2

In [ ]:
all_data = all_data.merge(kratnosti_2, how='outer', left_on='1', right_on='Субъект федерации')
all_data = all_data.drop('Субъект федерации',axis=1)

In [ ]:
dies_3 = pd.read_csv('data_oil_indexes/3_prices_petrolmap.csv', index_col=0, header=[0,1,2], parse_dates=True)

In [ ]:
dies_3 = dies_3[['Дизельное топливо летнее', 'Дизельное топливо зимнее']].resample('Y').mean().stack(2).mean(axis=1).unstack().transpose().iloc[:,[0,-2]]

In [ ]:
# dies_3

In [ ]:
dies_3.columns = ['Цены на дизель 2013', 'Цены на дизель 2018']

In [ ]:
maplist =  dies_3.index.values
rlist = all_data.iloc[:,1]

from fuzzywuzzy import fuzz

k = []
for i in maplist:
    lll = [fuzz.ratio(i,j) for j in rlist]
    k.append(np.argmax(lll))

rlist2 = rlist[k]

# pd.DataFrame([maplist, rlist2.values]).transpose().iloc[29:,:]

# rlist2.values

dies_3.index = rlist[k].values

dies_3.index.name = 'Субъект федерации'

# all_data

In [ ]:
all_data = all_data.merge(dies_3, how='outer', left_on='1', right_on='Субъект федерации')
# all_data = all_data.drop('Субъект федерации',axis=1)

In [ ]:
fuel_3 = pd.read_csv('data_oil_indexes/3_prices_fuel.csv', index_col=0, parse_dates=True).resample('Y').mean().transpose().iloc[:,[0,-2]]
fuel_3.columns = ['Цены на бензин 2013', 'Цены на бензин 2018']

In [ ]:
# all_data

In [ ]:
# fuel_3

In [ ]:
maplist =  fuel_3.index.values
rlist = all_data.iloc[:,1]

from fuzzywuzzy import fuzz

k = []
for i in maplist:
    lll = [fuzz.ratio(i,j) for j in rlist]
    k.append(np.argmax(lll))

rlist2 = rlist[k]

In [ ]:
# pd.DataFrame([maplist, rlist2.values]).transpose().iloc[29:,:]

In [ ]:
# rlist2.values

In [ ]:
fuel_3.index = rlist[k].values

In [ ]:
fuel_3.index.name = 'Субъект федерации'

In [ ]:
# all_data

In [ ]:
all_data = all_data.merge(fuel_3, how='outer', left_on='1', right_on='Субъект федерации')
# all_data = all_data.drop('Субъект федерации',axis=1)

In [ ]:
# all_data

In [ ]:
maz_3 = pd.read_csv('data_oil_indexes/3_prices_mazut.csv', index_col=0, parse_dates=True).resample('Y').mean().transpose().iloc[:,[0,-2]]
maz_3.columns = ['Цены на мазут 2012', 'Цены на мазут 2018']

crack_4 = pd.read_csv('data_oil_indexes/4_crack_spread_df.csv', index_col=0, parse_dates=True, header=[0,1]).transpose().swaplevel().transpose()['Крэк-спред'].resample('Y').mean().transpose().iloc[:,[0,-2]]
crack_4.columns = ['Крэк-спред 2013', 'Крэк-спред 2018']

akz_5 = pd.read_csv('data_oil_indexes/5_taxes_ndpi_2013_2017.csv', index_col=[0,1], parse_dates=True, header=0)['Акцизы на нефтепродукты  (суб.бюджет)'].unstack().iloc[:,[1,-1]].replace(0,np.nan) #.count()
akz_5.columns = ['Акцизы на нефтепродукты (суб.бюджет) 2014', 'Акцизы на нефтепродукты (суб.бюджет) 2017']

ndpin_5 = pd.read_csv('data_oil_indexes/5_taxes_ndpi_2013_2017.csv', index_col=[0,1], parse_dates=True, header=0)['НДПИ (нефть)'].unstack().iloc[:,[1,-1]].replace(0,np.nan)
ndpin_5.columns = ['НДПИ (нефть) 2014', 'НДПИ (нефть) 2017']

zadndpin_6 = pd.read_csv('data_oil_indexes/5_taxes_ndpi_2013_2017.csv', index_col=[0,1], parse_dates=True, header=0)['Задолженность по НДПИ'].unstack().iloc[:,[1,-1]].replace(0,np.nan)
zadndpin_6.columns = ['Задолженность по НДПИ 2014', 'Задолженность по НДПИ 2017']

zadakz_6 = pd.read_csv('data_oil_indexes/5_taxes_ndpi_2013_2017.csv', index_col=[0,1], parse_dates=True, header=0)['Задолженность по акцизу на нефтепродукты'].unstack().iloc[:,[1,-1]].replace(0,np.nan)
zadakz_6.columns = ['Задолженность по акцизу на нефтепродукты 2014', 'Задолженность по акцизу на нефтепродукты 2017']

zaddob_6 = pd.read_csv('data_oil_indexes/5_taxes_ndpi_2013_2017.csv', index_col=[0,1], parse_dates=True, header=0)['Задолженность по налогам и сборам в добыче сырой нефти и природного газа'].unstack().iloc[:,[1,-1]].replace(0,np.nan)
zaddob_6.columns = ['Задолженность по налогам и сборам в добыче сырой нефти и природного газа 2014', 'Задолженность по налогам и сборам в добыче сырой нефти и природного газа 2017']

ndgas_5 = pd.read_csv('data_oil_indexes/5_taxes_ndpi_2013_2017.csv', index_col=[0,1], parse_dates=True, header=0)['НДПИ (газ горючий природный)'].unstack().iloc[:,[1,-1]].replace(0,np.nan)
ndgas_5.columns = ['НДПИ (газ горючий природный) 2014', 'НДПИ (газ горючий природный) 2017']

ndcond_5 = pd.read_csv('data_oil_indexes/5_taxes_ndpi_2013_2017.csv', index_col=[0,1], parse_dates=True, header=0)['НДПИ (газовый конденсат )'].unstack().iloc[:,[1,-1]].replace(0,np.nan)
ndcond_5.columns = ['НДПИ (газовый конденсат ) 2014', 'НДПИ (газовый конденсат ) 2017']

In [ ]:
nals_56 = akz_5.merge(ndpin_5, how='outer', left_on='Субъект федерации', right_on='Субъект федерации')

In [ ]:
nals_56 = nals_56.merge(zadndpin_6, how='outer', left_on='Субъект федерации', right_on='Субъект федерации')
nals_56 = nals_56.merge(zadakz_6, how='outer', left_on='Субъект федерации', right_on='Субъект федерации')
nals_56 = nals_56.merge(zaddob_6, how='outer', left_on='Субъект федерации', right_on='Субъект федерации')
nals_56 = nals_56.merge(ndgas_5, how='outer', left_on='Субъект федерации', right_on='Субъект федерации')
nals_56 = nals_56.merge(ndcond_5, how='outer', left_on='Субъект федерации', right_on='Субъект федерации')


In [ ]:
cr_ind = crack_4.index.values

cr_ind[-9] = 'Республика Татарстан'

crack_4.index = cr_ind

crack_4.index.name='Субъект федерации'

In [ ]:
nals_4567 = nals_56.merge(crack_4, how='outer', left_on='Субъект федерации', right_on='Субъект федерации')


In [ ]:
spark_7 = pd.read_csv('data_oil_indexes/6_spark_electricity.csv', index_col=[0,1,2]).groupby('Субъект федерации').mean().iloc[:,[0,-2]]
spark_7.columns = ['Спарк-спред 2012', 'Спарк-спред 2018']
# spark_7

In [ ]:
sp_ind = spark_7.index.values

sp_ind[-11] = 'г. Санкт-Петербург'

sp_ind[-4] = 'Ханты-Мансийский АО-Югра'

sp_ind[-1] = 'Ямало-Ненецкий АО'

spark_7.index = sp_ind
spark_7.index.name='Субъект федерации'

In [ ]:
nals_4567 = nals_4567.merge(spark_7, how='outer', left_on='Субъект федерации', right_on='Субъект федерации')

In [ ]:
# nals_4567.shape, 
nals_4567 = nals_4567.iloc[:-1,:]

In [ ]:
maz_3.index.name='Субъект федерации'

In [ ]:
maz_ind = maz_3.index.values

In [ ]:
maz_ind[-9] = 'Республика Татарстан'

maz_3.index = maz_ind

maz_3.index.name='Субъект федерации'

In [ ]:
nals_34567 = nals_4567.merge(maz_3, how='outer', left_on='Субъект федерации', right_on='Субъект федерации')

In [ ]:
# fuel_3.index.name='Субъект федерации'
# fuel_3

In [ ]:
# nals_4567.merge(fuel_3, how='outer', left_on='Субъект федерации', right_on='Субъект федерации')

In [ ]:
maplist =  nals_34567.index.values
rlist = all_data.iloc[:,1]

from fuzzywuzzy import fuzz

k = []
for i in maplist:
    lll = [fuzz.ratio(i,j) for j in rlist]
    k.append(np.argmax(lll))

rlist2 = rlist[k]

In [ ]:
# pd.DataFrame([maplist, rlist2.values]).transpose() #.iloc[29:,:]

In [ ]:
# nals_4567

In [ ]:
nals_34567.index = rlist[k].values

In [ ]:
nals_34567.index.name = 'Субъект федерации'

In [ ]:
all_data = all_data.merge(nals_34567, how='outer', left_on='1', right_on='Субъект федерации')
# all_data = all_data.drop('Субъект федерации',axis=1)

In [ ]:
dor_5 = pd.read_csv('data_oil_indexes/5_dor_funds.csv', index_col=0).iloc[:,[0,5]]

In [ ]:
maplist =  dor_5.index.values
rlist = all_data.iloc[:,1]

from fuzzywuzzy import fuzz

k = []
for i in maplist:
    lll = [fuzz.ratio(i,j) for j in rlist]
    k.append(np.argmax(lll))

rlist2 = rlist[k]

In [ ]:
# pd.DataFrame([maplist, rlist2.values]).transpose().iloc[29:,:]

In [ ]:
# rlist2.values

dor_5.index = rlist[k].values

dor_5.index.name = 'Субъект федерации'

In [ ]:
all_data = all_data.merge(dor_5, how='outer', left_on='1', right_on='Субъект федерации')
# all_data = all_data.drop('Субъект федерации',axis=1)

In [ ]:
all_data.to_csv('all_data_for_map.csv')

In [ ]:
all_data = pd.read_csv('all_data_for_map.csv', index_col=0)

In [ ]:
russia_data = russia_with_crimea.merge(all_data, how='outer', left_on='NL_NAME_1', right_on='1')

In [ ]:
russia_data = russia_data.drop(['0.1', '1'], axis=1)

In [ ]:
russia_data.head()

## Аналитические показатели для рынка нефти и нефтепродуктов

### [Глубина переработки](https://ru.wikipedia.org/wiki/%D0%93%D0%BB%D1%83%D0%B1%D0%B8%D0%BD%D0%B0_%D0%BF%D0%B5%D1%80%D0%B5%D1%80%D0%B0%D0%B1%D0%BE%D1%82%D0%BA%D0%B8_%D0%BD%D0%B5%D1%84%D1%82%D0%B8)

- [ПРОГНОЗ РАЗВИТИЯ ЭНЕРГЕТИКИ МИРА И РОССИИ 2019](https://energy.skolkovo.ru/downloads/documents/SEneC/Research/SKOLKOVO_EneC_Forecast_2019_Rus.pdf)
- [Нефтяная отрасль: итоги 2017 года и краткосрочные перспективы](http://ac.gov.ru/files/publication/a/15796.pdf)
- [Российская нефтепереработка: пути развития](http://www.oilreview.ru/index.php?032)
- [К оценке эффективности работы нефтеперерабатывающих заводов](https://cyberleninka.ru/article/v/k-otsenke-effektivnosti-raboty-neftepererabatyvayuschih-zavodov)
- [Нефть-2017: итоги,тенденции, прогнозы](http://www.ngv.ru/upload/iblock/939/93972e1f25fa9b33508ca6c348e8614b.pdf)




Налоговый маневр:

- [НАЛОГОВЫЙ МАНЕВР В НЕФТЯНОЙ ОТРАСЛИ (2017)](https://cyberleninka.ru/article/n/nalogovyy-manevr-v-neftyanoy-otrasli)
    - Налоговый маневр, проводимый в нефтяной отрасли с конца 2014 г., должен был решить проблемы,
связанные с неэффективностью отечественной нефтепереработки. Однако корректировка его первона-
чальных параметров в течение двух последующих лет с целью получения дополнительных средств в
бюджет создает демотивирующие условия для НПЗ по повышению глубины переработки и модерниза-
ции производства.
- [НАЛОГОВЫЙ МАНЕВР В НЕФТЯНОЙ ОТРАСЛИ: БЛАГОПРИЯТНЫЙ МОМЕНТ ДЛЯ ЗАВЕРШЕНИЯ (2018)](https://cyberleninka.ru/article/n/nalogovyy-manevr-v-neftyanoy-otrasli-blagopriyatnyy-moment-dlya-zaversheniya)
    - Налоговое реформирование нефтяной отрасли, начатое в конце 2014 г., приносит некоторые позитив-
ные результаты. В 2014–2017 гг. увеличена средняя глубина переработки нефти в России. Текущий мо-
мент, несмотря на сегодняшние дискуссии, является благоприятным для завершения маневра и создания
действенных стимулов к повышению эффективности работы отечественных нефтеперерабатывающих
предприятий. Основным риском на этом пути являются конкретные параметры реформы, в частности –
величина субсидии НПЗ, механизм ее распределения и сроки предоставления.
- [НАЛОГОВЫЙ МАНЕВР В НЕФТЯНОЙ ОТРАСЛИ: РИСКИ «РУЧНОГО РЕГУЛИРОВАНИЯ» (2019)](https://cyberleninka.ru/article/n/nalogovyi-manevr-v-neftyanoi-otrasli-riski-ruchnogo-regulirovaniya)
    - Срок действия соглашения между ФАС, Минэнерго России и нефтяными компаниями о заморозке цен
на бензин и дизельное топливо истек 1 июля 2019 г. Предполагается, что в дальнейшем рост цен в
пределах инфляции должен будет обеспечивать демпфирующий механизм, параметры которого до
конца не определены. Затягивание с принятием решения по этому вопросу, тем более на фоне потенци-
ального ухудшения макроэкономических условий, может создать риск укоренения практики «ручного
регулирования» и заморозки цен 1 .
- [ЗАМОРОЗКА ЦЕН НА НЕФТЕПРОДУКТЫ – ОСТАНОВКА НАЛОГОВОГО МАНЕВРА? (2019)](https://cyberleninka.ru/article/n/zamorozka-tsen-na-nefteprodukty-ostanovka-nalogovogo-manevra)
    - В ноябре 2018 г. между нефтяными компаниями, ФАС и Минэнерго России было заключено соглашение
о заморозке цен на нефтяное топливо до марта 2019 г., что может стать потенциальным препятствием для
реализации завершающих мер налогового регулирования в нефтяной отрасли. Текущая конфигурация
параметров налогового маневра в совокупности с заморозкой розничных цен на нефтепродукты не
является оптимальной с точки зрения выгод и издержек для всех участников рынка: государства, нефтя-
ных компаний, потребителей.

In [ ]:
russia_data_2 = russia_data.copy()

In [ ]:
# russia_data_2['Глубина переработки, %'] = russia_data_2['Глубина переработки, %'].fillna(0)

In [ ]:
# current_cmap = matplotlib.cm.get_cmap('viridis')
# current_cmap.set_bad(color='grey')

In [ ]:
viridis = cm.get_cmap('viridis', 256)
newcolors = viridis(np.linspace(1, 0, 256))
grey = np.array([192/256, 192/256, 192/256, 1])
newcolors[:1, :] = grey
newcmp = ListedColormap(newcolors)

[working with projections](https://residentmario.github.io/geoplot/user_guide/Working_with_Projections.html)

In [ ]:
# gplt.polyplot(russia_data_2);

In [ ]:
# gplt.polyplot(russia_data_2, projection=gcrs.AlbersEqualArea());

In [ ]:

# f, axarr = plt.subplots(1, 2, figsize=(40, 20), subplot_kw={
#     'projection': proj
# })

# gplt.polyplot(russia_data_2, projection=proj, ax=axarr[0])
# gplt.polyplot(russia_data_2, projection=proj, ax=axarr[1]);

In [ ]:
# from mpl_toolkits.axes_grid1 import make_axes_locatable, axes_size

In [ ]:
proj = gcrs.AlbersEqualArea(central_longitude=94, central_latitude=60)

gplt.choropleth(
    russia_data_2, hue='Глубина переработки, %',
    cmap=newcmp, linewidth=0.5, edgecolor='white', k=None, legend=True,  #legend_kwargs={'size':3},
    projection=proj
);

In [ ]:
russia_data_2['Глубина переработки, %'][russia_data_2['Глубина переработки, %'] > 95]

In [ ]:
russia_data_2['Глубина переработки, %'].mean()

In [ ]:
russia_data_2['Глубина переработки, %'][russia_data_2['Глубина переработки, %'] < 60]

In [ ]:
replce10 =  russia_data_2['Глубина переработки, %'].min() - (russia_data_2['Глубина переработки, %'].max() - russia_data_2['Глубина переработки, %'].min())*0.01 * 0.9
russia_data_2['Глубина переработки, %'] = russia_data_2['Глубина переработки, %'].fillna(replce10)
replce10

In [ ]:
proj = gcrs.AlbersEqualArea(central_longitude=94, central_latitude=60)

f, axarr = plt.subplots(1, 1, figsize=(100, 20), subplot_kw={
    'projection': proj
})

f.subplots_adjust(wspace=0)

gplt.choropleth(
    russia_data_2, hue='Глубина переработки, %',
    cmap=newcmp, linewidth=0.8, edgecolor='white', k=None,  
    projection=proj,  ax=axarr
)

sm = plt.cm.ScalarMappable(cmap=newcmp)
sm._A = []

cbar = plt.colorbar(sm,  fraction=0.003, pad = -0.04, ticks=[0, 0.5, 1]) #orientation='horizontal',
cbar.ax.set_yticklabels([ '50 %', '75 %', '100 %'])  
# cbar.set_label(r'$\zeta\ [s^{-1}]$', fontsize=16)
cbar.ax.tick_params(labelsize=40)

# gplt.choropleth(
#     russia_data_2, hue='Глубина переработки, %',
#     cmap=newcmp, linewidth=0.5, edgecolor='white', k=None,  legend = False,  #legend_kwargs={'size':3},
#     projection=proj,  ax=axarr[1] 
    
# )
# plt.savefig('foo1.png')

In [ ]:
russia_data_2.columns

In [ ]:
# import seaborn as sns
# cmap = sns.diverging_palette(130, 10)

In [ ]:
gplt.choropleth(
    russia_data_2, hue=colnameee,
    cmap=newcmp, linewidth=0.5, edgecolor='white', k=None, legend=True,  #legend_kwargs={'size':3},
    projection=proj
)

In [ ]:
# russia_data_2['Выход светлых нефтепродуктов, %'][russia_data_2['Выход светлых нефтепродуктов, %']<60]

In [ ]:
colnameee = 'Выход светлых нефтепродуктов, %'
m1 = russia_data_2[colnameee].max()
m0 = russia_data_2[colnameee].min()
m05 = (m1+m0)/2

In [ ]:
russia_data_2 = russia_data.copy()

In [ ]:
m0, m1, m05

In [ ]:
russia_data_2[russia_data_2[colnameee] > 68]

In [ ]:
russia_data_2[colnameee].mean()

In [ ]:
russia_data_2[russia_data_2[colnameee] < 55].loc[:,['NL_NAME_1', colnameee]]

In [ ]:
# import seaborn as sns
# cmap = sns.diverging_palette(130, 10)

In [ ]:
russia_data_2 = russia_data.copy()

In [ ]:
replce10 = russia_data_2[colnameee].min() - (russia_data_2[colnameee].max() - russia_data_2[colnameee].min()) * 0.01
russia_data_2[colnameee] = russia_data_2[colnameee].fillna(replce10)
replce10

In [ ]:
# russia_data_2.loc[0,colnameee] = replce10

In [ ]:
proj = gcrs.AlbersEqualArea(central_longitude=94, central_latitude=60)

f, axarr = plt.subplots(1, 1, figsize=(100, 20), subplot_kw={
    'projection': proj
})

f.subplots_adjust(wspace=0)

gplt.choropleth(
    russia_data_2, hue=colnameee,
    cmap=newcmp, linewidth=0.8, edgecolor='white', k=None,  
    projection=proj,  ax=axarr
)

sm = plt.cm.ScalarMappable(cmap=newcmp)
sm._A = []

cbar = plt.colorbar(sm,  fraction=0.003, pad = -0.04, ticks=[0, 0.5, 1]) #orientation='horizontal',
cbar.ax.set_yticklabels([ '50 %', '60 %', '70 %'])  
# cbar.set_label(r'$\zeta\ [s^{-1}]$', fontsize=16)
cbar.ax.tick_params(labelsize=40)

# gplt.choropleth(
#     russia_data_2, hue='Глубина переработки, %',
#     cmap=newcmp, linewidth=0.5, edgecolor='white', k=None,  legend = False,  #legend_kwargs={'size':3},
#     projection=proj,  ax=axarr[1] 
    
# )
# plt.savefig('foo1.png')

In [ ]:
russia_data_2.columns

In [ ]:
colnameee = 'Индекс Нельсона'

In [ ]:
russia_data_2[russia_data_2[colnameee] > 8]

In [ ]:
russia_data_2[colnameee].mean()

In [ ]:
russia_data_2[russia_data_2[colnameee] < 1.5]

In [ ]:
gplt.choropleth(
    russia_data_2, hue=colnameee,
    cmap=newcmp, linewidth=0.5, edgecolor='white', k=None, legend=True,  #legend_kwargs={'size':3},
    projection=proj
);

In [ ]:
# russia_data_2['Выход светлых нефтепродуктов, %'][russia_data_2['Выход светлых нефтепродуктов, %']<60]

In [ ]:
m1 = russia_data_2[colnameee].max()
m0 = russia_data_2[colnameee].min()
m05 = (m1+m0)/2

In [ ]:
replce10 = russia_data_2[colnameee].min() - (russia_data_2[colnameee].max() - russia_data_2[colnameee].min()) * 0.01
russia_data_2[colnameee] = russia_data_2[colnameee].fillna(replce10)
replce10

In [ ]:
proj = gcrs.AlbersEqualArea(central_longitude=94, central_latitude=60)

f, axarr = plt.subplots(1, 1, figsize=(100, 20), subplot_kw={
    'projection': proj
})

f.subplots_adjust(wspace=0)

gplt.choropleth(
    russia_data_2, hue=colnameee,
    cmap=newcmp, linewidth=0.8, edgecolor='white', k=None,  
    projection=proj,  ax=axarr
)

sm = plt.cm.ScalarMappable(cmap=newcmp)
sm._A = []

cbar = plt.colorbar(sm,  fraction=0.003, pad = -0.04, ticks=[0, 0.5, 1]) #orientation='horizontal',
cbar.ax.set_yticklabels([m0, m05,  m1])  
# cbar.set_label(r'$\zeta\ [s^{-1}]$', fontsize=16)
cbar.ax.tick_params(labelsize=40)

# gplt.choropleth(
#     russia_data_2, hue='Глубина переработки, %',
#     cmap=newcmp, linewidth=0.5, edgecolor='white', k=None,  legend = False,  #legend_kwargs={'size':3},
#     projection=proj,  ax=axarr[1] 
    
# )
# plt.savefig('foo1.png')

In [ ]:
russia_data_2.columns

In [ ]:
colnameee = 'ИК'

In [ ]:
russia_data_2[russia_data_2[colnameee] > 160]

In [ ]:
russia_data_2[colnameee].mean()

In [ ]:
russia_data_2[russia_data_2[colnameee] < 20]

In [ ]:
gplt.choropleth(
    russia_data_2, hue=colnameee,
    cmap=newcmp, linewidth=0.5, edgecolor='white', k=None, legend=True,  #legend_kwargs={'size':3},
    projection=proj
);

In [ ]:
# russia_data_2['Выход светлых нефтепродуктов, %'][russia_data_2['Выход светлых нефтепродуктов, %']<60]

In [ ]:
m1 = russia_data_2[colnameee].max()
m0 = russia_data_2[colnameee].min()
m05 = (m1+m0)/2

In [ ]:
m0, m05,  m1

In [ ]:
viridis = cm.get_cmap('viridis', 256)
newcolors = viridis(np.linspace(0, 1, 256))
grey = np.array([192/256, 192/256, 192/256, 1])
newcolors[:1, :] = grey
backcmp = ListedColormap(newcolors)

In [ ]:
replce10 = russia_data_2[colnameee].min() - (russia_data_2[colnameee].max() - russia_data_2[colnameee].min()) * 0.01
russia_data_2[colnameee] = russia_data_2[colnameee].fillna(replce10)
replce10

In [ ]:
proj = gcrs.AlbersEqualArea(central_longitude=94, central_latitude=60)

f, axarr = plt.subplots(1, 1, figsize=(100, 20), subplot_kw={
    'projection': proj
})

f.subplots_adjust(wspace=0)

gplt.choropleth(
    russia_data_2, hue=colnameee,
    cmap=backcmp, linewidth=0.8, edgecolor='white', k=None,  
    projection=proj,  ax=axarr
)

sm = plt.cm.ScalarMappable(cmap=newcmp)
sm._A = []

cbar = plt.colorbar(sm,  fraction=0.003, pad = -0.04, ticks=[0, 0.5, 1]) #orientation='horizontal',
cbar.ax.set_yticklabels([180, 90, 5])  
# cbar.set_label(r'$\zeta\ [s^{-1}]$', fontsize=16)
cbar.ax.tick_params(labelsize=40)

# gplt.choropleth(
#     russia_data_2, hue='Глубина переработки, %',
#     cmap=newcmp, linewidth=0.5, edgecolor='white', k=None,  legend = False,  #legend_kwargs={'size':3},
#     projection=proj,  ax=axarr[1] 
    
# )
# plt.savefig('foo1.png')

### Отношение между объемом запасов и объемом добычи.

- [ЗА 1970-2007: ДОБЫЧА НЕФТИ, ВКЛЮЧАЯ ГАЗОВЫЙ КОНДЕНСАТ, ПО СУБЪЕКТАМ РОССИЙСКОЙ ФЕДЕРАЦИИ](http://www.gks.ru/bgd/regl/b08_13/IssWWW.exe/Stg/d3/13-27.htm), [1970-2005](https://uisrussia.msu.ru/stat/Publications/Ejeg2006/Ejeg2006_13_/Ejeg2006_13_290.htm?lang=ru)
- [О СОСТОЯНИИ И ИСПОЛЬЗОВАНИИ МИНЕРАЛЬНО-СЫРЬЕВЫХ РЕСУРСОВ РОССИЙСКОЙ ФЕДЕРАЦИИ В 2015 ГОДУ](http://www.mnr.gov.ru/upload/iblock/5ef/2015_msr.pdf)
- [Use of Statistical Indicators to Measure Crude Oil and Natural Gas Reserves and Production ](http://www.upg-bulletin-se.ro/archive/2014-1/5.Lazar_Lazar.pdf)

- [Нефть-2017: итоги,тенденции, прогнозы](http://www.ngv.ru/upload/iblock/939/93972e1f25fa9b33508ca6c348e8614b.pdf)

- [Нефтегазовый комплекс России –2017. ](http://www.ipgg.sbras.ru/ru/files/orgunits/energyreport/petroleum-industry-russia-2017.pdf)

- [Мировые  тенденции  геологоразведочных работ на нефть и газ](https://www.imemo.ru/files/File/ru/conf/2018/21122018/PREZ/01-17-Milovidov.pdf)

Описание показателя:


Запасы нефти:

- [Дагестан](https://cyberleninka.ru/article/v/otsenka-eksportnogo-potentsiala-neftegazodobyvayuschey-otrasli-respubliki-dagestan)
- [Калмыкия](https://cyberleninka.ru/article/v/razvitie-neftegazovogo-kompleksa-respubliki-kalmykiya-sostoyanie-i-perspektivy)
- [Омская](http://mpr.omskportal.ru/ru/RegionalPublicAuthorities/executivelist/MPR/pravaya-kolonka/ecopasport/PageContent/0/body_files/file21/%D0%A0%D0%B0%D0%B7%D0%B4%D0%B5%D0%BB%207.%20%D0%9C%D0%98%D0%9D%D0%95%D0%A0%D0%90%D0%9B%D0%AC%D0%9D%D0%9E-%D0%A1%D0%AB%D0%A0%D0%AC%D0%95%D0%92%D0%AB%D0%95%20%D0%A0%D0%95%D0%A1%D0%A3%D0%A0%D0%A1%D0%AB.%20%D0%9C%D0%98%D0%9D%D0%95%D0%A0%D0%90%D0%9B%D0%AC%D0%9D%D0%9E-%D0%A1%D0%AB%D0%A0%D0%AC%D0%95%D0%92%D0%AB%D0%95%20%D0%A0%D0%95%D0%A1%D0%A3%D0%A0%D0%A1%D0%AB.%20%D0%9C%D0%98%D0%9D%D0%95%D0%A0%D0%90%D0%9B%D0%AC%D0%9D%D0%9E-%D0%A1%D0%AB%D0%A0%D0%AC%D0%95%D0%92%D0%AB%D0%95%20%D0%A0%D0%95%D0%A1%D0%A3%D0%A0%D0%A1%D0%AB.pdf)
- Ингушетия  — экологический паспорт
- [Крым](https://tass.ru/ekonomika/1094468)
- [Грозненские месторождения Роснефть](https://ru.wikipedia.org/wiki/%D0%A1%D0%BF%D0%B8%D1%81%D0%BE%D0%BA_%D0%BD%D0%B5%D1%84%D1%82%D1%8F%D0%BD%D1%8B%D1%85_%D0%BC%D0%B5%D1%81%D1%82%D0%BE%D1%80%D0%BE%D0%B6%D0%B4%D0%B5%D0%BD%D0%B8%D0%B9_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8)
- [Краснодар](http://geolike.ru/page/gl_3671.htm)
    - Еще в 2014 г Министерство экономики Краснодарского края сообщало, что в регионе запасы сырья извлекли на 98,5%.
- [Волгоградская область](http://www.vestnik.vsu.ru/pdf/geograph/2014/02/2014-02-06.pdf)
- [Калининградская область](https://gov39.ru/region/natural.php)
- [Кировская область](https://nature.kirovreg.ru/ecology/nedropolzovanie/mineralno-syrevaya-baza-kirovskoy-oblasti/)
- [Новосибирская](https://ksonline.ru/238048/neftyanaya-promyshlennost-v-novosibirskoj-oblasti-prodolzhaet-razvivatsya/)
- [Пензенская](https://catalogmineralov.ru/deposit/penzenskaya_oblast/)
- [Cахалинская](https://cyberleninka.ru/article/v/neftegazovye-proekty-sahalinskoy-oblasti)




In [ ]:
russia_data_2.columns

In [ ]:
colnameee = 'Кратность запасов нефти добыче'

In [ ]:
russia_data_2[colnameee][russia_data_2[colnameee]>200] = 200

In [ ]:
# russia_data_2[colnameee] = np.exp(russia_data_2[colnameee])

In [ ]:
gplt.choropleth(
    russia_data_2, hue=colnameee,
    cmap=newcmp, linewidth=0.5, edgecolor='white', k=None, legend=True,  #legend_kwargs={'size':3},
    projection=proj
);

In [ ]:
# russia_data_2['Выход светлых нефтепродуктов, %'][russia_data_2['Выход светлых нефтепродуктов, %']<60]

In [ ]:
m1 = russia_data_2[colnameee].max()
m0 = russia_data_2[colnameee].min()
m05 = (m1+m0)/2

In [ ]:
m0, m05,  m1

In [ ]:
replce10 = russia_data_2[colnameee].min() - (russia_data_2[colnameee].max() - russia_data_2[colnameee].min()) * 0.01
russia_data_2[colnameee] = russia_data_2[colnameee].fillna(replce10)
replce10

In [ ]:
proj = gcrs.AlbersEqualArea(central_longitude=94, central_latitude=60)

f, axarr = plt.subplots(1, 1, figsize=(100, 20), subplot_kw={
    'projection': proj
})

f.subplots_adjust(wspace=0)

gplt.choropleth(
    russia_data_2, hue=colnameee,
    cmap=newcmp, linewidth=0.8, edgecolor='white', k=None,  
    projection=proj,  ax=axarr
)

sm = plt.cm.ScalarMappable(cmap=newcmp)
sm._A = []

cbar = plt.colorbar(sm,  fraction=0.003, pad = -0.04, ticks=[0, 0.5, 1]) #orientation='horizontal',
cbar.ax.set_yticklabels([' 15', ' 100', ' > 200'])  
# cbar.set_label(r'$\zeta\ [s^{-1}]$', fontsize=16)
cbar.ax.tick_params(labelsize=40)

# gplt.choropleth(
#     russia_data_2, hue='Глубина переработки, %',
#     cmap=newcmp, linewidth=0.5, edgecolor='white', k=None,  legend = False,  #legend_kwargs={'size':3},
#     projection=proj,  ax=axarr[1] 
    
# )
# plt.savefig('foo1.png')

### Цены на нефть и нефтепродукты

- [Паспорт набора данных "Интерактивная карта цен на нефтепродукты в России"](http://fas.gov.ru/opendata/7703516539-petrolpricesmap)
    - [табло](https://public.tableau.com/profile/fasanalytics#!/vizhome/_1206/sheet0)

- [ЦБСД](http://www.gks.ru/dbscripts/cbsd/DBInet.cgi)


Here are some important factors that affect crude oil prices.

- Oil demand: Broadly affected by the state of the global economy, as more economic activity generally requires more energy. Recently, non-OECD (Organisation for Economic Co-operation and Development) nations have been consuming more and more oil as their economies grow and develop. Meanwhile, oil demand from OECD nations has been flat or falling because their economies are more mature and they’re further using other fuel sources such as renewables, nuclear, and natural gas.
-   Oil supply: Affected by worldwide oil production. In recent years, oil production in the United States and Canada has increased greatly, which has added stability to global oil supply because production from other sources such as the North Sea has begun to decline. Additionally, OPEC (Organization of Petroleum Exporting Countries) plays a major role in setting global oil supply, as the member countries produce a significant portion of the world’s global oil. OPEC members also periodically convene to discuss optimal pricing and supply.
-   Speculation: Speculation by market participants through instruments such as crude oil futures also plays a role in setting oil prices. For instance, regarding the extreme oil price volatility seen in 2008, when prices hit almost $ \$ $ 150 per barrel at points, economist Thomas Palley stated, “The actual behavior of oil prices is consistent with speculation. In June, oil prices leapt by $ \$ $ 11 in one day, and in July they fell back by $ \$ $ 15 in three days. Such volatility does not fit a fundamentals-driven market.”



In [ ]:
russia_data.columns

In [ ]:
russia_data_3 = russia_data.copy()

In [ ]:
# russia_data_3.iloc[:,16:]

In [ ]:
russia_data_3.iloc[:,16:] = russia_data_3.iloc[:,16:].div(russia_data_3.iloc[:,16:].mean(axis=0))

In [ ]:
viridis = cm.get_cmap('RdYlBu', 256)
newcolors = viridis(np.linspace(1, -1, 256))
grey = np.array([192/256, 192/256, 192/256, 1])
newcolors[:1, :] = grey
rbcmp = ListedColormap(newcolors)

In [ ]:
russia_data_3['Цены на бензин 2013']

In [ ]:
colnameee = 'Цены на бензин 2013'

In [ ]:
gplt.choropleth(
    russia_data_3, hue=colnameee,
    cmap=newcmp, linewidth=0.5, edgecolor='white', k=None, legend=True,  #legend_kwargs={'size':3},
    projection=proj
);

In [ ]:
# russia_data_2['Выход светлых нефтепродуктов, %'][russia_data_2['Выход светлых нефтепродуктов, %']<60]

In [ ]:
m1 = russia_data_2[colnameee].max()
m0 = russia_data_2[colnameee].min()
m05 = (m1+m0)/2

In [ ]:
m0, m05,  m1

In [ ]:
russia_data_2.columns

In [ ]:
colnameee = 'Цены на бензин 2013'

In [ ]:
gplt.choropleth(
    russia_data_2, hue=colnameee,
    cmap=rbcmp, linewidth=0.5, edgecolor='white', k=None, legend=True,  #legend_kwargs={'size':3},
    projection=proj
);

In [ ]:
russia_data_3[colnameee][russia_data_3[colnameee] > 1.3] = 1.3

In [ ]:
# russia_data_2['Выход светлых нефтепродуктов, %'][russia_data_2['Выход светлых нефтепродуктов, %']<60]

In [ ]:
m1 = russia_data_3[colnameee].max()
m0 = russia_data_3[colnameee].min()
m05 = (m1+m0)/2

In [ ]:
m0, m05,  m1

In [ ]:
colnameee = 'Цены на бензин 2018'

In [ ]:
gplt.choropleth(
    russia_data_2, hue=colnameee,
    cmap=rbcmp, linewidth=0.5, edgecolor='white', k=None, legend=True,  #legend_kwargs={'size':3},
    projection=proj
);

In [ ]:
# russia_data_2['Выход светлых нефтепродуктов, %'][russia_data_2['Выход светлых нефтепродуктов, %']<60]

In [ ]:
m1 = russia_data_3[colnameee].max()
m0 = russia_data_3[colnameee].min()
m05 = (m1+m0)/2

In [ ]:
m0, m05,  m1

In [ ]:
10/(10+25)

In [ ]:
# replce10 = russia_data_2[colnameee].min() - (russia_data_2[colnameee].max() - russia_data_2[colnameee].min()) * 0.01
# russia_data_2[colnameee] = russia_data_2[colnameee].fillna(replce10)
# replce10

In [ ]:
proj = gcrs.AlbersEqualArea(central_longitude=94, central_latitude=60)

f, axarr = plt.subplots(1, 2, figsize=(100, 20), subplot_kw={
    'projection': proj
})

f.subplots_adjust(wspace=-0.15)

gplt.choropleth(
    russia_data_3, hue='Цены на бензин 2013',
    cmap=rbcmp, linewidth=0.5, edgecolor='white', k=None,  
    projection=proj,  ax=axarr[0]
)


gplt.choropleth(
    russia_data_3, hue='Цены на бензин 2018',
    cmap=rbcmp, linewidth=0.5, edgecolor='white', k=None,  legend = False,  #legend_kwargs={'size':3},
    projection=proj,  ax=axarr[1] 
    
)

sm = plt.cm.ScalarMappable(cmap=rbcmp)
sm._A = []

cbar = plt.colorbar(sm, ax=axarr[0], fraction=0.005, pad = -0.05, ticks=[0, 0.28, 1]) #orientation='horizontal',
cbar.ax.set_yticklabels([' - 10 %', ' 0 %', ' > 25 %'])  
# cbar.set_label(r'$\zeta\ [s^{-1}]$', fontsize=16)
cbar.ax.tick_params(labelsize=40)


In [ ]:
russia_data.columns

In [ ]:
colnameee = 'Цены на мазут 2012'

In [ ]:
# russia_data_2[colnameee][colnameee]<60]

In [ ]:
m1 = russia_data_3[colnameee].max()
m0 = russia_data_3[colnameee].min()
m05 = (m1+m0)/2

In [ ]:
m0, m05,  m1

In [ ]:
gplt.choropleth(
    russia_data_2, hue=colnameee,
    cmap=rbcmp, linewidth=0.5, edgecolor='white', k=None, legend=True,  #legend_kwargs={'size':3},
    projection=proj
);

In [ ]:
russia_data_3[colnameee][russia_data_3[colnameee] < 0.61] = 0.6

In [ ]:
russia_data_3[colnameee][russia_data_3[colnameee] > 1.4] = 1.4

In [ ]:
# russia_data_2['Выход светлых нефтепродуктов, %'][russia_data_2['Выход светлых нефтепродуктов, %']<60]

In [ ]:
m1 = russia_data_3[colnameee].max()
m0 = russia_data_3[colnameee].min()
m05 = (m1+m0)/2

In [ ]:
m0, m05,  m1

In [ ]:
colnameee = 'Цены на мазут 2018'

In [ ]:
gplt.choropleth(
    russia_data_2, hue=colnameee,
    cmap=rbcmp, linewidth=0.5, edgecolor='white', k=None, legend=True,  #legend_kwargs={'size':3},
    projection=proj
);

In [ ]:
# russia_data_2['Выход светлых нефтепродуктов, %'][russia_data_2['Выход светлых нефтепродуктов, %']<60]

In [ ]:
russia_data_3[colnameee][russia_data_3[colnameee] < 0.61] = 0.6

In [ ]:
russia_data_3[colnameee][russia_data_3[colnameee] > 1.4] = 1.4

In [ ]:
m1 = russia_data_3[colnameee].max()
m0 = russia_data_3[colnameee].min()
m05 = (m1+m0)/2

In [ ]:
m0, m05,  m1

In [ ]:
10/(10+25)

In [ ]:
proj = gcrs.AlbersEqualArea(central_longitude=94, central_latitude=60)

f, axarr = plt.subplots(1, 2, figsize=(100, 20), subplot_kw={
    'projection': proj
})

f.subplots_adjust(wspace=-0.15)

gplt.choropleth(
    russia_data_3, hue='Цены на мазут 2012',
    cmap=rbcmp, linewidth=0.5, edgecolor='white', k=None,  
    projection=proj,  ax=axarr[0]
)



gplt.choropleth(
    russia_data_3, hue='Цены на мазут 2018',
    cmap=rbcmp, linewidth=0.5, edgecolor='white', k=None,  legend = False,  #legend_kwargs={'size':3},
    projection=proj,  ax=axarr[1] 
    
)



sm = plt.cm.ScalarMappable(cmap=rbcmp)
sm._A = []

cbar = plt.colorbar(sm, ax=axarr[0], fraction=0.005, pad = -0.05, ticks=[0, 0.3, 1]) #orientation='horizontal',
cbar.ax.set_yticklabels([' < - 40 %', ' 0 %', ' > 50 %'])  
# cbar.set_label(r'$\zeta\ [s^{-1}]$', fontsize=16)
cbar.ax.tick_params(labelsize=40);


In [ ]:
russia_data.columns

In [ ]:
colnameee = 'Цены на дизель 2013'

In [ ]:
# russia_data_2[colnameee][colnameee]<60]

In [ ]:
m1 = russia_data_3[colnameee].max()
m0 = russia_data_3[colnameee].min()
m05 = (m1+m0)/2

In [ ]:
m0, m05,  m1

In [ ]:
gplt.choropleth(
    russia_data, hue=colnameee,
    cmap=rbcmp, linewidth=0.5, edgecolor='white', k=None, legend=True,  #legend_kwargs={'size':3},
    projection=proj
);

In [ ]:
russia_data_3[colnameee][russia_data_3[colnameee] < 0.9] = 0.9

In [ ]:
russia_data_3[colnameee][russia_data_3[colnameee] > 1.4] = 1.4

In [ ]:
# russia_data_2['Выход светлых нефтепродуктов, %'][russia_data_2['Выход светлых нефтепродуктов, %']<60]

In [ ]:
m1 = russia_data_3[colnameee].max()
m0 = russia_data_3[colnameee].min()
m05 = (m1+m0)/2

In [ ]:
m0, m05,  m1

In [ ]:
colnameee = 'Цены на дизель 2018'

In [ ]:
gplt.choropleth(
    russia_data_3, hue=colnameee,
    cmap=rbcmp, linewidth=0.5, edgecolor='white', k=None, legend=True,  #legend_kwargs={'size':3},
    projection=proj
);

In [ ]:
# russia_data_2['Выход светлых нефтепродуктов, %'][russia_data_2['Выход светлых нефтепродуктов, %']<60]

In [ ]:
russia_data_3[colnameee][russia_data_3[colnameee] < 0.61] = 0.6

In [ ]:
russia_data_3[colnameee][russia_data_3[colnameee] > 1.4] = 1.4

In [ ]:
m1 = russia_data_3[colnameee].max()
m0 = russia_data_3[colnameee].min()
m05 = (m1+m0)/2

In [ ]:
m0, m05,  m1

In [ ]:
10/(10+25)

In [ ]:
proj = gcrs.AlbersEqualArea(central_longitude=94, central_latitude=60)

f, axarr = plt.subplots(1, 2, figsize=(100, 20), subplot_kw={
    'projection': proj
})

f.subplots_adjust(wspace=-0.15)

gplt.choropleth(
    russia_data_3, hue='Цены на дизель 2013',
    cmap=rbcmp, linewidth=0.5, edgecolor='white', k=None,  
    projection=proj,  ax=axarr[0]
)



gplt.choropleth(
    russia_data_3, hue='Цены на дизель 2018',
    cmap=rbcmp, linewidth=0.5, edgecolor='white', k=None,  legend = False,  #legend_kwargs={'size':3},
    projection=proj,  ax=axarr[1] 
    
)



sm = plt.cm.ScalarMappable(cmap=rbcmp)
sm._A = []

cbar = plt.colorbar(sm, ax=axarr[0], fraction=0.005, pad = -0.05, ticks=[0, 0.2, 1]) #orientation='horizontal',
cbar.ax.set_yticklabels([' < - 10 %', ' 0 %', ' > 40 %'])  
# cbar.set_label(r'$\zeta\ [s^{-1}]$', fontsize=16)
cbar.ax.tick_params(labelsize=40);


### Крэк-спред

- [СПРЕДЫ ПЕРЕРАБОТКИ: Крек-спред](http://www.fondovik.com/bazovie-znaniya/stati-finansovie-instrumenti/fyuchersnie-spredi/krek-spred/)
- [EIA: ПРИБЫЛИ НПЗ НЕДОСТАТОЧНО ДЛЯ КОМПЕНСАЦИИ НИЗКИХ ДОХОДОВ MAJORS](http://www.ngv.ru/analytics/eia_pribyli_npz_nedostatochno_dlya_kompensatsii_nizkikh_dokhodov_majors/)
- [EIA: crack-spread definition](https://www.eia.gov/todayinenergy/includes/crackspread_explain.php )
- [Crack Spread 101](https://marketrealist.com/2013/07/crack-spread-101-part-1-whats-a-crack-spread/)

We use the example of the “Bloomberg WTI Cushing Crude Oil 321 Crack Spread/Gulf Coast” (Bloomberg ticker CRKS321 Index). Bloomberg notes that the inputs are the following:

- Crude oil – WTI at Cushing (USCRWTIC Index ) – closing price at July 19, 2013, of \$ 108.05 per barrel
- Gasoline – U.S. Gulf prompt 87 Octane gasoline (MOIGC87P Index) – closing price at July 19, 2013, of \$2.92 per gallon (multiply by 42 to get price per barrel of \$122.64)
- Heating oil or gasoil – U.S. Gulf No. 2 Oil (NO2IGCPR index) – closing price at July 19, 2013, of \$2.87 per gallon (multiply by 42 to get price per barrel of \$120.54)

Цены на мазут:

- [ТОПЛИВНЫЙ  РЫНОК  РОССИИ](http://web.archive.org/web/20110725123304/http://minenergo.gov.ru/upload/iblock/333/33363d868cf93701591957ccd67583b1.pdf)
- [КОРТЕС](http://www.kortes.com/products/price/)
    - [РЕГИОНАЛЬНЫЕ ОПТОВЫЕ ЦЕНЫ НА ОСНОВНЫЕ НЕФТЕПРОДУКТЫ ](http://www.kortes.com/products/pdf/reg.pdf)
- [ЦБСД Индексы по нефтепереработке](http://www.gks.ru/free_doc/new_site/prices/bd/bd_9420059.htm)

Цены на нефть:

- [Индексы цен нефтепродуктов](http://spimex.com/documents/indexes/)
    - [Методика расчета региональных биржевых индексов цен сырой нефти](http://spimex.com/upload/iblock/9b0/9b06acc0ae4d4f48eaedaa5473554cf0.pdf)
- [Коэффициенты перевода литров в тонны:](http://bogatoe.samregion.ru/external/omsu_bogatoe/files/c_683/Perevod_benzina_v_tonny.pdf)

In [ ]:
viridis = cm.get_cmap('RdYlBu', 256)
newcolors = viridis(np.linspace(-1, 1, 256))
grey = np.array([192/256, 192/256, 192/256, 1])
newcolors[:1, :] = grey
bcrbcmp = ListedColormap(newcolors)

In [ ]:
russia_data.columns

In [ ]:
colnameee = 'Крэк-спред 2013'

In [ ]:
# russia_data_2[colnameee][colnameee]<60]

In [ ]:
m1 = russia_data_2[colnameee].max()
m0 = russia_data_2[colnameee].min()
m05 = (m1+m0)/2

In [ ]:
m0, m05,  m1

In [ ]:
gplt.choropleth(
    russia_data, hue=colnameee,
    cmap=rbcmp, linewidth=0.5, edgecolor='white', k=None, legend=True,  #legend_kwargs={'size':3},
    projection=proj
);

In [ ]:
russia_data_2[colnameee][russia_data_2[colnameee] > 19000] =  19000

In [ ]:
russia_data_2[colnameee][russia_data_2[colnameee] < 16000]
# = 1.4

In [ ]:
# russia_data_2['Выход светлых нефтепродуктов, %'][russia_data_2['Выход светлых нефтепродуктов, %']<60]

In [ ]:
m1 = russia_data_3[colnameee].max()
m0 = russia_data_3[colnameee].min()
m05 = (m1+m0)/2

In [ ]:
m0, m05,  m1

In [ ]:
replce10 = russia_data_2[colnameee].min() - (russia_data_2[colnameee].max() - russia_data_2[colnameee].min()) * 0.01
russia_data_2[colnameee] = russia_data_2[colnameee].fillna(replce10)
replce10

In [ ]:
colnameee = 'Крэк-спред 2018'

In [ ]:
gplt.choropleth(
    russia_data, hue=colnameee,
    cmap=rbcmp, linewidth=0.5, edgecolor='white', k=None, legend=True,  #legend_kwargs={'size':3},
    projection=proj
);

In [ ]:
# russia_data_2['Выход светлых нефтепродуктов, %'][russia_data_2['Выход светлых нефтепродуктов, %']<60]

In [ ]:
russia_data_2[colnameee][russia_data_2[colnameee] > 19000] = 19000

In [ ]:
russia_data_2[colnameee][russia_data_2[colnameee] < 15000] = 15000

In [ ]:
m1 = russia_data_2[colnameee].max()
m0 = russia_data_2[colnameee].min()
m05 = (m1+m0)/2

In [ ]:
m0, m05,  m1

In [ ]:
replce10 = russia_data_2[colnameee].min() - (russia_data_2[colnameee].max() - russia_data_2[colnameee].min()) * 0.01
russia_data_2[colnameee] = russia_data_2[colnameee].fillna(replce10)
replce10

In [ ]:
proj = gcrs.AlbersEqualArea(central_longitude=94, central_latitude=60)

f, axarr = plt.subplots(1, 2, figsize=(100, 20), subplot_kw={
    'projection': proj
})

f.subplots_adjust(wspace=-0.15)

gplt.choropleth(
    russia_data_3, hue='Крэк-спред 2013',
    cmap=newcmp, linewidth=0.5, edgecolor='white', k=None,  
    projection=proj,  ax=axarr[0]
)



gplt.choropleth(
    russia_data_3, hue='Крэк-спред 2018',
    cmap=newcmp, linewidth=0.5, edgecolor='white', k=None,  legend = False,  #legend_kwargs={'size':3},
    projection=proj,  ax=axarr[1] 
    
)



sm = plt.cm.ScalarMappable(cmap=newcmp)
sm._A = []

cbar = plt.colorbar(sm, ax=axarr[0], fraction=0.005, pad = -0.05, ticks=[0, 0.5, 1]) #orientation='horizontal',
cbar.ax.set_yticklabels([' < 15,000', '  17,000', ' > 19,000'])  
# cbar.set_label(r'$\zeta\ [s^{-1}]$', fontsize=16)
cbar.ax.tick_params(labelsize=40);


### Объемы собираемых налоговых платежей (НДПИ, акциз) в региональном разрезе

- [НДПИ](https://ru.wikipedia.org/wiki/%D0%9D%D0%B0%D0%BB%D0%BE%D0%B3_%D0%BD%D0%B0_%D0%B4%D0%BE%D0%B1%D1%8B%D1%87%D1%83_%D0%BF%D0%BE%D0%BB%D0%B5%D0%B7%D0%BD%D1%8B%D1%85_%D0%B8%D1%81%D0%BA%D0%BE%D0%BF%D0%B0%D0%B5%D0%BC%D1%8B%D1%85_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D1%8F)
- [СТАВКИ АКЦИЗА НА НЕФТЬ И ГАЗОВЫЙ КОНДЕНСАТ ДЛЯ НЕФТЕГАЗОДОБЫВАЮЩИХ ПРЕДПРИЯТИЙ](https://zakonbase.ru/content/part/196311)

-  [Об Оценке влияния налОгОвОгО маневра в нефтянОй сфере на пОтребительскую инфляцию](http://www.cbr.ru/Content/Document/File/59793/20181227.pdf)

In [ ]:
russia_data.columns

In [ ]:
colnameee = 'НДПИ (нефть) 2014'

In [ ]:
# russia_data_2[colnameee][colnameee]<60]

In [ ]:
m1 = russia_data_2[colnameee].max()
m0 = russia_data_2[colnameee].min()
m05 = (m1+m0)/2

In [ ]:
m0, m05,  m1

In [ ]:
gplt.choropleth(
    russia_data_2, hue=colnameee,
    cmap=rbcmp, linewidth=0.5, edgecolor='white', k=None, legend=True,  #legend_kwargs={'size':3},
    projection=proj
);

In [ ]:
russia_data_2[colnameee][russia_data_2[colnameee] == 150000] = 250000
# =  19000

In [ ]:
russia_data_2[colnameee][russia_data_2[colnameee] < 16000]
# = 1.4

In [ ]:
# russia_data_2['Выход светлых нефтепродуктов, %'][russia_data_2['Выход светлых нефтепродуктов, %']<60]

In [ ]:
m1 = russia_data_3[colnameee].max()
m0 = russia_data_3[colnameee].min()
m05 = (m1+m0)/2

In [ ]:
m0, m05,  m1

In [ ]:
russia_data_2 = russia_data.copy()

In [ ]:
replce10 = russia_data_2[colnameee].min() - (russia_data_2[colnameee].max() - russia_data_2[colnameee].min()) * 0.01
russia_data_2[colnameee] = russia_data_2[colnameee].fillna(replce10)
replce10

In [ ]:
colnameee = 'НДПИ (нефть) 2017'

In [ ]:
gplt.choropleth(
    russia_data, hue=colnameee,
    cmap=rbcmp, linewidth=0.5, edgecolor='white', k=None, legend=True,  #legend_kwargs={'size':3},
    projection=proj
);

In [ ]:
# russia_data_2['Выход светлых нефтепродуктов, %'][russia_data_2['Выход светлых нефтепродуктов, %']<60]

In [ ]:
russia_data_2[colnameee][russia_data_2[colnameee] > 250000] = 250000
# = 19000

In [ ]:
russia_data_2[colnameee][russia_data_2[colnameee] < 15000] = 15000

In [ ]:
m1 = russia_data_2[colnameee].max()
m0 = russia_data_2[colnameee].min()
m05 = (m1+m0)/2

In [ ]:
m0, m05,  m1

In [ ]:
replce10 = russia_data_2[colnameee].min() - (russia_data_2[colnameee].max() - russia_data_2[colnameee].min()) * 0.01
russia_data_2[colnameee] = russia_data_2[colnameee].fillna(replce10)
replce10

In [ ]:
proj = gcrs.AlbersEqualArea(central_longitude=94, central_latitude=60)

f, axarr = plt.subplots(1, 2, figsize=(100, 20), subplot_kw={
    'projection': proj
})

f.subplots_adjust(wspace=-0.15)

gplt.choropleth(
    russia_data_2, hue='НДПИ (нефть) 2014',
    cmap=newcmp, linewidth=0.5, edgecolor='white', k=None,  
    projection=proj,  ax=axarr[0]
)

sm = plt.cm.ScalarMappable(cmap=newcmp)
sm._A = []

cbar = plt.colorbar(sm, ax=axarr[0], fraction=0.005, pad = -0.05, ticks=[0, 0.5, 1]) #orientation='horizontal',
cbar.ax.set_yticklabels([' 0', ' 125,000', '> 250,000'])  
# cbar.set_label(r'$\zeta\ [s^{-1}]$', fontsize=16)
cbar.ax.tick_params(labelsize=40)



gplt.choropleth(
    russia_data_2, hue='НДПИ (нефть) 2017',
    cmap=newcmp, linewidth=0.5, edgecolor='white', k=None,  legend = False,  #legend_kwargs={'size':3},
    projection=proj,  ax=axarr[1] 
    
);






In [ ]:
russia_data.columns

In [ ]:
colnameee = 'Акцизы на нефтепродукты (суб.бюджет) 2014'

In [ ]:
# russia_data_2[colnameee][colnameee]<60]

In [ ]:
m1 = russia_data_2[colnameee].max()
m0 = russia_data_2[colnameee].min()
m05 = (m1+m0)/2

In [ ]:
m0, m05,  m1

In [ ]:
gplt.choropleth(
    russia_data_2, hue=colnameee,
    cmap=rbcmp, linewidth=0.5, edgecolor='white', k=None, legend=True,  #legend_kwargs={'size':3},
    projection=proj
);

In [ ]:
russia_data_2[colnameee][russia_data_2[colnameee] > 30000] = 30000
# = 250000
# =  19000

In [ ]:
russia_data_2[colnameee][russia_data_2[colnameee] < -300] = -300
# = 1.4

In [ ]:
# russia_data_2['Выход светлых нефтепродуктов, %'][russia_data_2['Выход светлых нефтепродуктов, %']<60]

In [ ]:
m1 = russia_data_3[colnameee].max()
m0 = russia_data_3[colnameee].min()
m05 = (m1+m0)/2

In [ ]:
m0, m05,  m1

In [ ]:
replce10 = russia_data_2[colnameee].min() - (russia_data_2[colnameee].max() - russia_data_2[colnameee].min()) * 0.01
russia_data_2[colnameee] = russia_data_2[colnameee].fillna(-350)
replce10

In [ ]:
colnameee = 'Акцизы на нефтепродукты (суб.бюджет) 2017'

In [ ]:
gplt.choropleth(
    russia_data, hue=colnameee,
    cmap=rbcmp, linewidth=0.5, edgecolor='white', k=None, legend=True,  #legend_kwargs={'size':3},
    projection=proj
);

In [ ]:
# russia_data_2['Выход светлых нефтепродуктов, %'][russia_data_2['Выход светлых нефтепродуктов, %']<60]

In [ ]:
russia_data_2[colnameee][russia_data_2[colnameee] > 30000] = 30000
# = 19000

In [ ]:
russia_data_2[colnameee][russia_data_2[colnameee] < -300] = -300
# = 15000

In [ ]:
m1 = russia_data_2[colnameee].max()
m0 = russia_data_2[colnameee].min()
m05 = (m1+m0)/2

In [ ]:
m0, m05,  m1

In [ ]:
replce10 = russia_data_2[colnameee].min() - (russia_data_2[colnameee].max() - russia_data_2[colnameee].min()) * 0.01
russia_data_2[colnameee] = russia_data_2[colnameee].fillna(-350)
replce10

In [ ]:
proj = gcrs.AlbersEqualArea(central_longitude=94, central_latitude=60)

f, axarr = plt.subplots(1, 2, figsize=(100, 20), subplot_kw={
    'projection': proj
})

f.subplots_adjust(wspace=-0.15)

gplt.choropleth(
    russia_data_2, hue='Акцизы на нефтепродукты (суб.бюджет) 2014',
    cmap=newcmp, linewidth=0.5, edgecolor='white', k=None,  
    projection=proj,  ax=axarr[0]
)

sm = plt.cm.ScalarMappable(cmap=newcmp)
sm._A = []

cbar = plt.colorbar(sm, ax=axarr[0], fraction=0.005, pad = -0.05, ticks=[0, 0.5, 1]) #orientation='horizontal',
cbar.ax.set_yticklabels([' < - 300', '  15,000', ' > 30,000'])  
# cbar.set_label(r'$\zeta\ [s^{-1}]$', fontsize=16)
cbar.ax.tick_params(labelsize=40)



gplt.choropleth(
    russia_data_2, hue='Акцизы на нефтепродукты (суб.бюджет) 2017',
    cmap=newcmp, linewidth=0.5, edgecolor='white', k=None,  legend = False,  #legend_kwargs={'size':3},
    projection=proj,  ax=axarr[1] 
    
);






In [ ]:
russia_data.columns

In [ ]:
colnameee = 'Акцизы на нефтепродукты (факт. 2016 г. )'

In [ ]:
# russia_data_2[colnameee][colnameee]<60]

In [ ]:
m1 = russia_data_2[colnameee].max()
m0 = russia_data_2[colnameee].min()
m05 = (m1+m0)/2

In [ ]:
m0, m05,  m1

In [ ]:
gplt.choropleth(
    russia_data_2, hue=colnameee,
    cmap=rbcmp, linewidth=0.5, edgecolor='white', k=None, legend=True,  #legend_kwargs={'size':3},
    projection=proj
);

In [ ]:
russia_data_2[colnameee][russia_data_2[colnameee] > 16000] = 16000
# =  19000

In [ ]:
russia_data_2[colnameee][russia_data_2[colnameee] < 16000]
# = 1.4

In [ ]:
# russia_data_2['Выход светлых нефтепродуктов, %'][russia_data_2['Выход светлых нефтепродуктов, %']<60]

In [ ]:
m1 = russia_data_3[colnameee].max()
m0 = russia_data_3[colnameee].min()
m05 = (m1+m0)/2

In [ ]:
m0, m05,  m1

In [ ]:
colnameee = 'Акцизы на нефтепродукты (Прогноз 2017 г.)'

In [ ]:
gplt.choropleth(
    russia_data, hue=colnameee,
    cmap=rbcmp, linewidth=0.5, edgecolor='white', k=None, legend=True,  #legend_kwargs={'size':3},
    projection=proj
);

In [ ]:
# russia_data_2['Выход светлых нефтепродуктов, %'][russia_data_2['Выход светлых нефтепродуктов, %']<60]

In [ ]:
russia_data_2[colnameee][russia_data_2[colnameee] > 16000] = 16000
# = 19000

In [ ]:
russia_data_2[colnameee][russia_data_2[colnameee] < 100] = 0
# = 0

In [ ]:
m1 = russia_data_2[colnameee].max()
m0 = russia_data_2[colnameee].min()
m05 = (m1+m0)/2

In [ ]:
m0, m05,  m1

In [ ]:
(30+46)/2

In [ ]:
viridis = cm.get_cmap('viridis', 256)
newcolors = viridis(np.linspace(1, 0, 256))
# grey = np.array([192/256, 192/256, 192/256, 1])
# newcolors[:1, :] = grey
newcmp = ListedColormap(newcolors)

In [ ]:
proj = gcrs.AlbersEqualArea(central_longitude=94, central_latitude=60)

f, axarr = plt.subplots(1, 2, figsize=(100, 20), subplot_kw={
    'projection': proj
})

f.subplots_adjust(wspace=-0.15)

gplt.choropleth(
    russia_data_2, hue='Акцизы на нефтепродукты (факт. 2016 г. )',
    cmap=newcmp, linewidth=0.5, edgecolor='white', k=None,  
    projection=proj,  ax=axarr[0]
)

sm = plt.cm.ScalarMappable(cmap=newcmp)
sm._A = []

cbar = plt.colorbar(sm, ax=axarr[0], fraction=0.005, pad = -0.05, ticks=[0, 0.5, 1]) #orientation='horizontal',
cbar.ax.set_yticklabels([' 0', ' 8,000', ' 16,000'])  
# cbar.set_label(r'$\zeta\ [s^{-1}]$', fontsize=16)
cbar.ax.tick_params(labelsize=40)



gplt.choropleth(
    russia_data_2, hue='Акцизы на нефтепродукты (Прогноз 2017 г.)',
    cmap=newcmp, linewidth=0.5, edgecolor='white', k=None,  legend = False,  #legend_kwargs={'size':3},
    projection=proj,  ax=axarr[1] 
    
);






In [ ]:
viridis = cm.get_cmap('viridis', 256)
newcolors = viridis(np.linspace(1, 0, 256))
grey = np.array([192/256, 192/256, 192/256, 1])
newcolors[:1, :] = grey
newcmp = ListedColormap(newcolors)

### Задолженность по налоговым платежам (НДПИ, акциз) в региональном разрезе

- [ВИЗУАЛИЗАЦИЯ ДАННЫХ, ХАРАКТЕРИЗУЮЩИХ НАЛОГОВУЮ КУЛЬТУРУ В НЕФТЕГАЗОВОМ СЕКТОРЕ](https://wciom.ru/fileadmin/file/monitoring/2016/134/2016_134_10_Karibekova.pdf)

In [ ]:
russia_data.columns

In [ ]:
colnameee = 'Задолженность по НДПИ 2014'

In [ ]:
# russia_data_2[colnameee][colnameee]<60]

In [ ]:
m1 = russia_data_2[colnameee].max()
m0 = russia_data_2[colnameee].min()
m05 = (m1+m0)/2

In [ ]:
m0, m05,  m1

In [ ]:
gplt.choropleth(
    russia_data_2, hue=colnameee,
    cmap=rbcmp, linewidth=0.5, edgecolor='white', k=None, legend=True,  #legend_kwargs={'size':3},
    projection=proj
);

In [ ]:
russia_data_2[colnameee][russia_data_2[colnameee] > 500] = 500
# = 30000
# = 250000
# =  19000

In [ ]:
russia_data_2[colnameee][russia_data_2[colnameee] < -300] = -300
# = 1.4

In [ ]:
# russia_data_2['Выход светлых нефтепродуктов, %'][russia_data_2['Выход светлых нефтепродуктов, %']<60]

In [ ]:
m1 = russia_data_3[colnameee].max()
m0 = russia_data_3[colnameee].min()
m05 = (m1+m0)/2

In [ ]:
m0, m05,  m1

In [ ]:
colnameee = 'Задолженность по НДПИ 2017'

In [ ]:
gplt.choropleth(
    russia_data, hue=colnameee,
    cmap=rbcmp, linewidth=0.5, edgecolor='white', k=None, legend=True,  #legend_kwargs={'size':3},
    projection=proj
);

In [ ]:
# russia_data_2['Выход светлых нефтепродуктов, %'][russia_data_2['Выход светлых нефтепродуктов, %']<60]

In [ ]:
russia_data_2[colnameee][russia_data_2[colnameee] > 500] = 500
# = 19000

In [ ]:
russia_data_2[colnameee][russia_data_2[colnameee] < -300] = -300
# = 15000

In [ ]:
m1 = russia_data_2[colnameee].max()
m0 = russia_data_2[colnameee].min()
m05 = (m1+m0)/2

In [ ]:
m0, m05,  m1

In [ ]:
(30+46)/2

In [ ]:
proj = gcrs.AlbersEqualArea(central_longitude=94, central_latitude=60)

f, axarr = plt.subplots(1, 2, figsize=(100, 20), subplot_kw={
    'projection': proj
})

f.subplots_adjust(wspace=-0.15)

gplt.choropleth(
    russia_data_2, hue='Задолженность по НДПИ 2014',
    cmap=newcmp, linewidth=0.5, edgecolor='white', k=None,  
    projection=proj,  ax=axarr[0]
)

sm = plt.cm.ScalarMappable(cmap=newcmp)
sm._A = []

cbar = plt.colorbar(sm, ax=axarr[0], fraction=0.005, pad = -0.05, ticks=[0, 0.5, 1]) #orientation='horizontal',
cbar.ax.set_yticklabels([' 0', '  250', ' > 500'])  
# cbar.set_label(r'$\zeta\ [s^{-1}]$', fontsize=16)
cbar.ax.tick_params(labelsize=40)



gplt.choropleth(
    russia_data_2, hue='Задолженность по НДПИ 2017',
    cmap=newcmp, linewidth=0.5, edgecolor='white', k=None,  legend = False,  #legend_kwargs={'size':3},
    projection=proj,  ax=axarr[1] 
    
);






In [ ]:
russia_data.columns

In [ ]:
colnameee = 'Задолженность по акцизу на нефтепродукты 2014'

In [ ]:
# russia_data_2[colnameee][colnameee]<60]

In [ ]:
m1 = russia_data_2[colnameee].max()
m0 = russia_data_2[colnameee].min()
m05 = (m1+m0)/2

In [ ]:
m0, m05,  m1

In [ ]:
gplt.choropleth(
    russia_data_2, hue=colnameee,
    cmap=rbcmp, linewidth=0.5, edgecolor='white', k=None, legend=True,  #legend_kwargs={'size':3},
    projection=proj
);

In [ ]:
russia_data_2[colnameee][russia_data_2[colnameee] > 500] = 500
# = 30000
# = 250000
# =  19000

In [ ]:
russia_data_2[colnameee][russia_data_2[colnameee].isna()] = -5

In [ ]:
russia_data_2[colnameee][russia_data_2[colnameee] < -300] = -300
# = 1.4

In [ ]:
# russia_data_2['Выход светлых нефтепродуктов, %'][russia_data_2['Выход светлых нефтепродуктов, %']<60]

In [ ]:
m1 = russia_data_3[colnameee].max()
m0 = russia_data_3[colnameee].min()
m05 = (m1+m0)/2

In [ ]:
m0, m05,  m1

In [ ]:
colnameee = 'Задолженность по акцизу на нефтепродукты 2017'

In [ ]:
gplt.choropleth(
    russia_data, hue=colnameee,
    cmap=rbcmp, linewidth=0.5, edgecolor='white', k=None, legend=True,  #legend_kwargs={'size':3},
    projection=proj
);

In [ ]:
# russia_data_2['Выход светлых нефтепродуктов, %'][russia_data_2['Выход светлых нефтепродуктов, %']<60]

In [ ]:
russia_data_2[colnameee][russia_data_2[colnameee] > 500] = 500
# = 19000

In [ ]:
russia_data_2[colnameee][russia_data_2[colnameee] < -300] = -300
# = 15000

In [ ]:
m1 = russia_data_2[colnameee].max()
m0 = russia_data_2[colnameee].min()
m05 = (m1+m0)/2

In [ ]:
m0, m05,  m1

In [ ]:
(30+46)/2

In [ ]:
proj = gcrs.AlbersEqualArea(central_longitude=94, central_latitude=60)

f, axarr = plt.subplots(1, 2, figsize=(100, 20), subplot_kw={
    'projection': proj
})

f.subplots_adjust(wspace=-0.15)

gplt.choropleth(
    russia_data_2, hue='Задолженность по акцизу на нефтепродукты 2014',
    cmap=newcmp, linewidth=0.5, edgecolor='white', k=None,  
    projection=proj,  ax=axarr[0]
)

sm = plt.cm.ScalarMappable(cmap=newcmp)
sm._A = []

cbar = plt.colorbar(sm, ax=axarr[0], fraction=0.005, pad = -0.05, ticks=[0, 0.5, 1]) #orientation='horizontal',
cbar.ax.set_yticklabels([' 0', '  250', ' > 500'])  
# cbar.set_label(r'$\zeta\ [s^{-1}]$', fontsize=16)
cbar.ax.tick_params(labelsize=40)



gplt.choropleth(
    russia_data_2, hue='Задолженность по акцизу на нефтепродукты 2017',
    cmap=newcmp, linewidth=0.5, edgecolor='white', k=None,  legend = False,  #legend_kwargs={'size':3},
    projection=proj,  ax=axarr[1] 
    
);






In [ ]:
russia_data.columns

In [ ]:
colnameee = 'Задолженность по налогам и сборам в добыче сырой нефти и природного газа 2014'

In [ ]:
# russia_data_2[colnameee][colnameee]<60]
russia_data_2[colnameee][russia_data_2[colnameee].isna()] = -5

In [ ]:
m1 = russia_data_2[colnameee].max()
m0 = russia_data_2[colnameee].min()
m05 = (m1+m0)/2

In [ ]:
m0, m05,  m1

In [ ]:
gplt.choropleth(
    russia_data_2, hue=colnameee,
    cmap=rbcmp, linewidth=0.5, edgecolor='white', k=None, legend=True,  #legend_kwargs={'size':3},
    projection=proj
);

In [ ]:
russia_data_2[colnameee][russia_data_2[colnameee] > 800] = 1000
# = 30000
# = 19000
# =  19000

In [ ]:
russia_data_2[colnameee][russia_data_2[colnameee] < -300] = -300
# = 1.4

In [ ]:
# russia_data_2['Выход светлых нефтепродуктов, %'][russia_data_2['Выход светлых нефтепродуктов, %']<60]

In [ ]:
m1 = russia_data_3[colnameee].max()
m0 = russia_data_3[colnameee].min()
m05 = (m1+m0)/2

In [ ]:
m0, m05,  m1

In [ ]:
colnameee = 'Задолженность по налогам и сборам в добыче сырой нефти и природного газа 2017'

russia_data_2[colnameee][russia_data_2[colnameee].isna()] = -5

In [ ]:
gplt.choropleth(
    russia_data, hue=colnameee,
    cmap=rbcmp, linewidth=0.5, edgecolor='white', k=None, legend=True,  #legend_kwargs={'size':3},
    projection=proj
);

In [ ]:
# russia_data_2['Выход светлых нефтепродуктов, %'][russia_data_2['Выход светлых нефтепродуктов, %']<60]

In [ ]:
russia_data_2[colnameee][russia_data_2[colnameee] > 800] = 1000
# = 30000
# = 19000

In [ ]:
russia_data_2[colnameee][russia_data_2[colnameee] < -300] = -300
# = 15000

In [ ]:
m1 = russia_data_2[colnameee].max()
m0 = russia_data_2[colnameee].min()
m05 = (m1+m0)/2

In [ ]:
m0, m05,  m1

In [ ]:
(30+46)/2

In [ ]:
proj = gcrs.AlbersEqualArea(central_longitude=94, central_latitude=60)

f, axarr = plt.subplots(1, 2, figsize=(100, 20), subplot_kw={
    'projection': proj
})

f.subplots_adjust(wspace=-0.15)

gplt.choropleth(
    russia_data_2, hue='Задолженность по налогам и сборам в добыче сырой нефти и природного газа 2014',
    cmap=newcmp, linewidth=0.5, edgecolor='white', k=None,  
    projection=proj,  ax=axarr[0]
)

sm = plt.cm.ScalarMappable(cmap=newcmp)
sm._A = []

cbar = plt.colorbar(sm, ax=axarr[0], fraction=0.005, pad = -0.05, ticks=[0, 0.5, 1]) #orientation='horizontal',
cbar.ax.set_yticklabels([' < - 300', '  15,000', ' > 30,000'])  
# cbar.set_label(r'$\zeta\ [s^{-1}]$', fontsize=16)
cbar.ax.tick_params(labelsize=40)



gplt.choropleth(
    russia_data_2, hue='Задолженность по налогам и сборам в добыче сырой нефти и природного газа 2017',
    cmap=newcmp, linewidth=0.5, edgecolor='white', k=None,  legend = False,  #legend_kwargs={'size':3},
    projection=proj,  ax=axarr[1] 
    
);






## Аналитические показатели для рынка газа

### Отношение между объемом запасов и объемом добычи в региональном разрезе

- [Use of Statistical Indicators to Measure Crude Oil and Natural Gas Reserves and Production ](http://www.upg-bulletin-se.ro/archive/2014-1/5.Lazar_Lazar.pdf)

-  [Воспроизводство минерально-сырьевой базы нефтегазового комплекса России](https://cyberleninka.ru/article/v/vosproizvodstvo-mineralno-syrievoy-bazy-neftegazovogo-kompleksa-rossii)


In [ ]:
russia_data_2.columns

In [ ]:
colnameee = 'Кратность запасов газа добыче'

In [ ]:
russia_data_2[colnameee][russia_data_2[colnameee]>1000] = 1000

In [ ]:
# russia_data_2[colnameee] = np.exp(russia_data_2[colnameee])

In [ ]:
gplt.choropleth(
    russia_data_2, hue=colnameee,
    cmap=newcmp, linewidth=0.5, edgecolor='white', k=None, legend=True,  #legend_kwargs={'size':3},
    projection=proj
);

In [ ]:
# russia_data_2['Выход светлых нефтепродуктов, %'][russia_data_2['Выход светлых нефтепродуктов, %']<60]

In [ ]:
m1 = russia_data_2[colnameee].max()
m0 = russia_data_2[colnameee].min()
m05 = (m1+m0)/2

In [ ]:
m0, m05,  m1

In [ ]:
# russia_data.co
russia_data_2 = russia_data.copy()

In [ ]:
replce10 = russia_data_2[colnameee].min() - (russia_data_2[colnameee].max() - russia_data_2[colnameee].min()) * 0.01
russia_data_2[colnameee] = russia_data_2[colnameee].fillna(0)
replce10

In [ ]:
proj = gcrs.AlbersEqualArea(central_longitude=94, central_latitude=60)

f, axarr = plt.subplots(1, 1, figsize=(100, 20), subplot_kw={
    'projection': proj
})

f.subplots_adjust(wspace=0)

gplt.choropleth(
    russia_data_2, hue=colnameee,
    cmap=newcmp, linewidth=0.8, edgecolor='white', k=None,  
    projection=proj,  ax=axarr
)

sm = plt.cm.ScalarMappable(cmap=newcmp)
sm._A = []

cbar = plt.colorbar(sm,  fraction=0.003, pad = -0.04, ticks=[0, 0.5, 1]) #orientation='horizontal',
cbar.ax.set_yticklabels([' 25', ' 500', ' > 1000'])  
# cbar.set_label(r'$\zeta\ [s^{-1}]$', fontsize=16)
cbar.ax.tick_params(labelsize=40)

# gplt.choropleth(
#     russia_data_2, hue='Глубина переработки, %',
#     cmap=newcmp, linewidth=0.5, edgecolor='white', k=None,  legend = False,  #legend_kwargs={'size':3},
#     projection=proj,  ax=axarr[1] 
    
# )
# plt.savefig('foo1.png')

### Объемы собираемых налоговых платежей (НДПИ) в региональном разрезе

- [О ставках акциза на природный газ
](http://pravo.gov.ru/proxy/ips/?docbody=&nd=102057751&rdk=1)

Акциз не  уплачивается организациями, передающими   или
реализующими на  территории  Российской  Федерации  природный  газ,
приобретенный   и  реализованный  по  государственным  регулируемым
оптовым ценам,  акциз на который уже  был  уплачен  поставщиками  в
установленном    порядке    исходя    из   налогооблагаемой   базы,
предусмотренной в абзаце втором настоящего пункта.

In [ ]:
russia_data.columns

In [ ]:
colnameee = 'НДПИ (газ горючий природный) 2014'

In [ ]:
# russia_data_2[colnameee][colnameee]<60]

In [ ]:
m1 = russia_data_2[colnameee].max()
m0 = russia_data_2[colnameee].min()
m05 = (m1+m0)/2

In [ ]:
m0, m05,  m1

In [ ]:
gplt.choropleth(
    russia_data_2, hue=colnameee,
    cmap=rbcmp, linewidth=0.5, edgecolor='white', k=None, legend=True,  #legend_kwargs={'size':3},
    projection=proj
);

In [ ]:
russia_data_2[colnameee][russia_data_2[colnameee] > 20000] = 20000
# = 250000
# =  19000

In [ ]:
russia_data_2[colnameee][russia_data_2[colnameee] < 16000]
# = 1.4

In [ ]:
# russia_data_2['Выход светлых нефтепродуктов, %'][russia_data_2['Выход светлых нефтепродуктов, %']<60]

In [ ]:
m1 = russia_data_3[colnameee].max()
m0 = russia_data_3[colnameee].min()
m05 = (m1+m0)/2

In [ ]:
m0, m05,  m1

In [ ]:
colnameee = 'НДПИ (газ горючий природный) 2017'

In [ ]:
gplt.choropleth(
    russia_data, hue=colnameee,
    cmap=rbcmp, linewidth=0.5, edgecolor='white', k=None, legend=True,  #legend_kwargs={'size':3},
    projection=proj
);

In [ ]:
# russia_data_2['Выход светлых нефтепродуктов, %'][russia_data_2['Выход светлых нефтепродуктов, %']<60]

In [ ]:
russia_data_2[colnameee][russia_data_2[colnameee] > 20000] = 20000
# = 19000

In [ ]:
russia_data_2[colnameee][russia_data_2[colnameee] < 15000] = 15000

In [ ]:
m1 = russia_data_2[colnameee].max()
m0 = russia_data_2[colnameee].min()
m05 = (m1+m0)/2

In [ ]:
m0, m05,  m1

In [ ]:
(30+46)/2

In [ ]:
proj = gcrs.AlbersEqualArea(central_longitude=94, central_latitude=60)

f, axarr = plt.subplots(1, 2, figsize=(100, 20), subplot_kw={
    'projection': proj
})

f.subplots_adjust(wspace=-0.15)

gplt.choropleth(
    russia_data_2, hue='НДПИ (газ горючий природный) 2014',
    cmap=newcmp, linewidth=0.5, edgecolor='white', k=None,  
    projection=proj,  ax=axarr[0]
)

sm = plt.cm.ScalarMappable(cmap=newcmp)
sm._A = []

cbar = plt.colorbar(sm, ax=axarr[0], fraction=0.005, pad = -0.05, ticks=[0, 0.5, 1]) #orientation='horizontal',
cbar.ax.set_yticklabels([' 0', ' 10,000', '> 20,000'])  
# cbar.set_label(r'$\zeta\ [s^{-1}]$', fontsize=16)
cbar.ax.tick_params(labelsize=40)



gplt.choropleth(
    russia_data_2, hue='НДПИ (газ горючий природный) 2017',
    cmap=newcmp, linewidth=0.5, edgecolor='white', k=None,  legend = False,  #legend_kwargs={'size':3},
    projection=proj,  ax=axarr[1] 
    
);






In [ ]:
russia_data.columns

In [ ]:
colnameee = 'НДПИ (газовый конденсат ) 2014'

In [ ]:
# russia_data_2[colnameee][colnameee]<60]

In [ ]:
m1 = russia_data_2[colnameee].max()
m0 = russia_data_2[colnameee].min()
m05 = (m1+m0)/2

In [ ]:
m0, m05,  m1

In [ ]:
gplt.choropleth(
    russia_data_2, hue=colnameee,
    cmap=rbcmp, linewidth=0.5, edgecolor='white', k=None, legend=True,  #legend_kwargs={'size':3},
    projection=proj
);

In [ ]:
russia_data_2[colnameee][russia_data_2[colnameee] > 3000] = 3000
# = 250000
# =  19000

In [ ]:
russia_data_2[colnameee][russia_data_2[colnameee] < 16000]
# = 1.4

In [ ]:
# russia_data_2['Выход светлых нефтепродуктов, %'][russia_data_2['Выход светлых нефтепродуктов, %']<60]

In [ ]:
m1 = russia_data_3[colnameee].max()
m0 = russia_data_3[colnameee].min()
m05 = (m1+m0)/2

In [ ]:
m0, m05,  m1

In [ ]:
colnameee = 'НДПИ (газовый конденсат ) 2017'

In [ ]:
gplt.choropleth(
    russia_data, hue=colnameee,
    cmap=rbcmp, linewidth=0.5, edgecolor='white', k=None, legend=True,  #legend_kwargs={'size':3},
    projection=proj
);

In [ ]:
# russia_data_2['Выход светлых нефтепродуктов, %'][russia_data_2['Выход светлых нефтепродуктов, %']<60]

In [ ]:
russia_data_2[colnameee][russia_data_2[colnameee] > 3000] = 3000
# = 19000

In [ ]:
russia_data_2[colnameee][russia_data_2[colnameee] < 15000] = 15000

In [ ]:
m1 = russia_data_2[colnameee].max()
m0 = russia_data_2[colnameee].min()
m05 = (m1+m0)/2

In [ ]:
m0, m05,  m1

In [ ]:
(30+46)/2

In [ ]:
proj = gcrs.AlbersEqualArea(central_longitude=94, central_latitude=60)

f, axarr = plt.subplots(1, 2, figsize=(100, 20), subplot_kw={
    'projection': proj
})

f.subplots_adjust(wspace=-0.15)

gplt.choropleth(
    russia_data_2, hue='НДПИ (газовый конденсат ) 2014',
    cmap=newcmp, linewidth=0.5, edgecolor='white', k=None,  
    projection=proj,  ax=axarr[0]
)

sm = plt.cm.ScalarMappable(cmap=newcmp)
sm._A = []

cbar = plt.colorbar(sm, ax=axarr[0], fraction=0.005, pad = -0.05, ticks=[0, 0.5, 1]) #orientation='horizontal',
cbar.ax.set_yticklabels([' 0', ' 1,500', '> 3,000'])  
# cbar.set_label(r'$\zeta\ [s^{-1}]$', fontsize=16)
cbar.ax.tick_params(labelsize=40)



gplt.choropleth(
    russia_data_2, hue='НДПИ (газовый конденсат ) 2017',
    cmap=newcmp, linewidth=0.5, edgecolor='white', k=None,  legend = False,  #legend_kwargs={'size':3},
    projection=proj,  ax=axarr[1] 
    
);






## Аналитические показатели для рынка электроэнергетики

- [ГИС энергосбережение](https://gisee.ru/regionsupport/)

### Спарк-спред

Генераторы, работающие на других видах топлива, имеют аналогичные показатели -- дарк-спред (цена на электроэнергию за вычетом стоимости угля) для угольных генераторов, кварк-спред (цена на электроэнергию за вычетом стоимости обогащенного уранового топлива) для ядерных генераторов. Основная цель использования этих спредов – моделирование  экономики производства электроэнергии. 


1 мегаватт равно 0.859845 Гигакалорий в час

https://www.eia.gov/todayinenergy/detail.php?id=9911

- [Спарк-спред](http://www.fondovik.com/bazovie-znaniya/stati-finansovie-instrumenti/fyuchersnie-spredi/spark-spred/)

- [РСВ на покупку ЭЭ](http://www.atsenergo.ru/results/rsv/statistics?period=3) -  Рынок на сутки вперед (РСВ) 
    - [Участникам оптового рынка](http://www.atsenergo.ru/uchastnikam-optovogo-rynka)
- [Тарифы на электроэнергию для населения в России](https://energybase.ru/tariff/electricity)
- [Тарифы на электроэнергию по субъектам Российской Федерации на 2018 год в рублях за кВт*ч (сводная таблица)](https://worknet-info.ru/read-blog/1142_%D1%82%D0%B0%D0%B1%D0%BB%D0%B8%D1%86%D0%B0-%D1%82%D0%B0%D1%80%D0%B8%D1%84%D0%BE%D0%B2-%D0%BD%D0%B0-%D1%8D%D0%BB%D0%B5%D0%BA%D1%82%D1%80%D0%BE%D1%8D%D0%BD%D0%B5%D1%80%D0%B3%D0%B8%D1%8E-%D0%BF%D0%BE-%D1%80%D0%B5%D0%B3%D0%B8%D0%BE%D0%BD%D0%B0%D0%BC-%D1%80%D0%BE%D1%81%D1%81%D0%B8%D0%B8-%D0%B2-2018-%D0%B3%D0%BE%D0%B4%D1%83.html)

[Heat rate](https://en.wikipedia.org/wiki/Heat_rate_(efficiency):

- [Heat Rate](https://data.austintexas.gov/stories/s/Heat-Rate/h4dw-35vc/)
- [NPHR formula](https://www.powermag.com/understanding-coal-power-plant-heat-rate-and-efficiency/?pagenum=3)
- [Heat Rate = Input/Otput](https://www.myodesie.com/wiki/index/returnEntry/id/2995)
- [Определение энергетической эффективности аппаратов, установок и систем](http://www.energosovet.ru/stat616.html)

- [Сравнительные характеристики ТЭС разного типа и АЭС](https://lesprominform.ru/jarticles.html?id=3418)

- [Сравнение различных видов топлива](http://biopechi.ru/stati/sravnenie-razlichnykh-vidov-topliva/)

- [нормативы](https://zakonbase.ru/content/part/613281)

- [Средний удельный расход - топливо](https://www.ngpedia.ru/id380315p1.html)

- [Экономия ушла в полезный отпуск ](https://rg.ru/2019/03/04/rashod-topliva-pri-vyrabotke-elektroenergii-v-rossii-snizilsia.html)

руб за 1000 м3 = руб за  7 900 000  ккал

руб за 1000 м3 = руб за  7 900 000  000 кал

3000 за 7 900 000 000
x за 1 000 000 000

х = цена * 1 000 000 000 / 7 900 000 000

х = цена * 1 000   / 7 900  

7900  ккал/м3  *  руб за 1000 м3

Собрать цены на уголь (среднее по углям??)

- http://www.gks.ru/dbscripts/cbsd/

In [ ]:
russia_data.columns

In [ ]:
colnameee = 'Спарк-спред 2012'

In [ ]:
# russia_data_2[colnameee][colnameee]<60]

In [ ]:
m1 = russia_data_2[colnameee].max()
m0 = russia_data_2[colnameee].min()
m05 = (m1+m0)/2

In [ ]:
m0, m05,  m1

In [ ]:
gplt.choropleth(
    russia_data, hue=colnameee,
    cmap=rbcmp, linewidth=0.5, edgecolor='white', k=None, legend=True,  #legend_kwargs={'size':3},
    projection=proj
);

In [ ]:
russia_data_2[colnameee][russia_data_2[colnameee] > 19000] 
# =  19000

In [ ]:
russia_data_2[colnameee][russia_data_2[colnameee] < 16000]
# = 1.4

In [ ]:
# russia_data_2['Выход светлых нефтепродуктов, %'][russia_data_2['Выход светлых нефтепродуктов, %']<60]

In [ ]:
m1 = russia_data_3[colnameee].max()
m0 = russia_data_3[colnameee].min()
m05 = (m1+m0)/2

In [ ]:
m0, m05,  m1

In [ ]:
colnameee = 'Спарк-спред 2018'

In [ ]:
gplt.choropleth(
    russia_data, hue=colnameee,
    cmap=rbcmp, linewidth=0.5, edgecolor='white', k=None, legend=True,  #legend_kwargs={'size':3},
    projection=proj
);

In [ ]:
# russia_data_2['Выход светлых нефтепродуктов, %'][russia_data_2['Выход светлых нефтепродуктов, %']<60]

In [ ]:
russia_data_2[colnameee][russia_data_2[colnameee] > 19000] 
# = 19000

In [ ]:
russia_data_2[colnameee][russia_data_2[colnameee] < 15000] = 15000

In [ ]:
m1 = russia_data_2[colnameee].max()
m0 = russia_data_2[colnameee].min()
m05 = (m1+m0)/2

In [ ]:
m0, m05,  m1

In [ ]:
(30+46)/2

In [ ]:
proj = gcrs.AlbersEqualArea(central_longitude=94, central_latitude=60)

f, axarr = plt.subplots(1, 2, figsize=(100, 20), subplot_kw={
    'projection': proj
})

f.subplots_adjust(wspace=-0.15)

gplt.choropleth(
    russia_data_3, hue='Спарк-спред 2012',
    cmap=newcmp, linewidth=0.5, edgecolor='white', k=None,  
    projection=proj,  ax=axarr[0]
)

sm = plt.cm.ScalarMappable(cmap=newcmp)
sm._A = []

cbar = plt.colorbar(sm, ax=axarr[0], fraction=0.005, pad = -0.05, ticks=[0, 0.5, 1]) #orientation='horizontal',
cbar.ax.set_yticklabels([' 1,600', '  2,250', ' 2,900'])  
# cbar.set_label(r'$\zeta\ [s^{-1}]$', fontsize=16)
cbar.ax.tick_params(labelsize=40)



gplt.choropleth(
    russia_data_3, hue='Спарк-спред 2018',
    cmap=newcmp, linewidth=0.5, edgecolor='white', k=None,  legend = False,  #legend_kwargs={'size':3},
    projection=proj,  ax=axarr[1] 
    
)





sm = plt.cm.ScalarMappable(cmap=newcmp)
sm._A = []

cbar = plt.colorbar(sm, ax=axarr[1], fraction=0.005, pad = -0.05, ticks=[0, 0.5, 1]) #orientation='horizontal',
cbar.ax.set_yticklabels([' 3,000', ' 3,800', ' 4,600'])  
# cbar.set_label(r'$\zeta\ [s^{-1}]$', fontsize=16)
cbar.ax.tick_params(labelsize=40);
